<a href="https://colab.research.google.com/github/rybread1/stack-overflow-question-classification/blob/master/stack_overflow_questions_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 890kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 32.3MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
     |████████████████████████████████| 3.0MB 17.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f4b9139d32e5be3810aaf2de2e9ac6bc7de6ca900d2f347f00d09c576623c684
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import math
import seaborn as sn
import tensorflow as tf
import tensorflow_datasets as tfds
from bs4 import BeautifulSoup
from sklearn import metrics as sk_metrics
import nltk
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
df = pd.read_csv('/content/gdrive/My Drive/stack-overflow-data.csv', encoding='utf-8')

# encode all of the target variables: HQ --> 0, 1 --> LQ_CLOSE, 2 --> LQ_EDIT
df['Y_cat'] = df['Y'].astype('category').cat.codes
# extract the hour from the creation date
df['CreationDateTime'] = pd.to_datetime(df['CreationDate']).dt.hour
# concat the title and the body
# df['Text'] = df['Body'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)
df['title_body'] = df['Title'] + ' ' + df['Body']

In [5]:
df.sample(10)

,Id,Title,Body,Tags,CreationDate,Y,Y_cat,CreationDateTime,title_body
8687,37531205,check if a string starts with number using reg...,<p>I am writing a filebeat configuration when ...,<regex><logstash><filebeat>,2016-05-30 17:45:06,HQ,0,17,check if a string starts with number using reg...
28725,44544052,PHP for loop increment by 100000,<p>I want to loop through <code>$i</code> whic...,<php><for-loop>,2017-06-14 11:54:17,LQ_CLOSE,1,11,PHP for loop increment by 100000 <p>I want to ...
44363,50999977,What does the gather function do in pytorch in...,"<p>I have been through the <a href=""https://py...",<pytorch>,2018-06-23 09:54:01,HQ,0,9,What does the gather function do in pytorch in...
32706,46092104,Subclass in type hinting,<p>I want to allow type hinting using Python 3...,<python><subclass><type-hinting>,2017-09-07 08:59:09,HQ,0,8,Subclass in type hinting <p>I want to allow ty...
6400,36642952,visual composer not showing on my webiste,Yesterday I worded on visual composer and it w...,<wordpress>,2016-04-15 09:12:55,LQ_EDIT,2,9,visual composer not showing on my webiste Yest...
6119,36482782,Python coding explanation,<p>I am having a hard time understanding this ...,<python>,2016-04-07 17:04:01,LQ_CLOSE,1,17,Python coding explanation <p>I am having a har...
32721,46100272,Azure Portal: Bad Request - Request Too Long,<p>I just received the following error when I ...,<azure-active-directory><azure-ad-b2c><azurepo...,2017-09-07 15:34:32,HQ,0,15,Azure Portal: Bad Request - Request Too Long <...
29521,44817161,Installing docker on azure virtual machine win...,<p>I am getting an error upon installing docke...,<azure><docker><windows-10><hyper-v>,2017-06-29 05:44:08,HQ,0,5,Installing docker on azure virtual machine win...
6062,36467150,"When using conemu with Windows 10 bash shell, ...",<p>The up arrow scrolls through command histor...,<windows><bash><conemu>,2016-04-07 05:09:08,HQ,0,5,"When using conemu with Windows 10 bash shell, ..."
28616,44496789,How can I randomize an array without repeating...,"<p>I have 3 arrays ( questions, answers and co...",<java><android>,2017-06-12 10:07:00,LQ_CLOSE,1,10,How can I randomize an array without repeating...


# Data Preprocessing

*   remove anything other than white spaces brackets and a-z characters.
*   lowercase the next
*   remove common stop words


In [6]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def data_cleaning(data):
    data = data.lower()
    data = re.sub(r'([^a-zA-Z\s])', '', data)
    data = data.split()
    temp = []
    for i in data:
        if i not in stop_words:
            temp.append(i)
    data = ' '.join(temp)
    return data

y = np.array(df['Y_cat'], dtype=np.int64)
x = (str(df['CreationDateTime']) + ' ' + df['title_body'].apply(lambda x: data_cleaning(x))).values

train_size = int(df.shape[0] * 0.8)
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Build a LSTM model from scratch

In [25]:
# keras tokenizer
MAX_WORDS = 20000

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=MAX_WORDS, 
    oov_token=MAX_WORDS+1,
    
    )
# fit on training data
tokenizer.fit_on_texts(x_train)

# convert text to tokenized sequences
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

# add padding
x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_tokens, maxlen=200, padding='post')
x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test_tokens, maxlen=200, padding='post')

# initiate tf datasets
ds_train = tf.data.Dataset.from_tensor_slices((x_train_padded, y_train)).batch(64)
ds_test = tf.data.Dataset.from_tensor_slices((x_test_padded, y_test)).batch(64)

# model parameters
embedding_dim = 512
vocab_size = MAX_WORDS + 2
n_classes = df['Y_cat'].nunique()

# build the model
inputs = tf.keras.Input(shape=(None,), dtype="int32")
x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# call bacls
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=2, 
    min_lr=0.0001)

# fit the model
results = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=100,
    callbacks=[early_stop_callback, reduce_lr]
)

# BERT Tokenizer

In [7]:
from tokenizers import BertWordPieceTokenizer
import transformers

In [8]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower=True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [9]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=200):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [10]:
max_len=200

x_train = fast_encode(x_train, fast_tokenizer, maxlen=max_len)
x_test = fast_encode(x_test, fast_tokenizer, maxlen=max_len)

In [11]:
bert_transformer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
sequence_output = bert_transformer(input_word_ids)[0]
cls_token = sequence_output[:, 0, :]
out = tf.keras.layers.Dense(3, activation='softmax')(cls_token)

model = tf.keras.Model(inputs=input_word_ids, outputs=out)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=7e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 200, 768),)       66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 66,365,187
Trainable params: 66,365,187
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    validation_data=(x_test,y_test),
    epochs=5)

Epoch 1/5
3000/3000 [==============================] - 1182s 394ms/step - loss: 0.2925 - accuracy: 0.8684 - val_loss: 0.4803 - val_accuracy: 0.7956
Epoch 2/5
3000/3000 [==============================] - 1176s 392ms/step - loss: 0.1751 - accuracy: 0.9285 - val_loss: 0.2554 - val_accuracy: 0.8948
Epoch 3/5
3000/3000 [==============================] - 1178s 393ms/step - loss: 0.1424 - accuracy: 0.9433 - val_loss: 0.1862 - val_accuracy: 0.9259
Epoch 4/5
3000/3000 [==============================] - 1177s 392ms/step - loss: 0.1135 - accuracy: 0.9553 - val_loss: 0.2498 - val_accuracy: 0.9099
Epoch 5/5
3000/3000 [==============================] - 1178s 393ms/step - loss: 0.0824 - accuracy: 0.9692 - val_loss: 0.4440 - val_accuracy: 0.8627
